In [1]:
!pip install music21

In [2]:
from music21 import converter,note,chord,stream,instrument
import glob
import pickle
import numpy as np
from keras.utils import np_utils
import os
from keras.models import Sequential,load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from keras.optimizers import Adam

TensorFlow 1.x selected.


Using TensorFlow backend.


## Reading notes file

In [0]:
with open('./drive/My Drive/PreprocessedData/notes','rb') as f:
    notes = pickle.load(f)
f.close()

In [20]:
n_vocab = len(set(notes))
print(len(notes))
print(n_vocab)

65730
381


## Generating the sequential data for the LSTM model

In [0]:
snotes = sorted(set(notes))

In [0]:
ele_int = dict((ele,idx) for idx,ele in enumerate(snotes))

In [0]:
sequence_length = 50

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length] # in the string format
    seq_out = notes[i+sequence_length]
    
    network_input.append([ele_int[ch] for ch in seq_in])
    network_output.append(ele_int[seq_out])

In [26]:
print(len(network_input))
n_patterns = len(network_input)
print(network_input[0])

65680
[333, 352, 330, 333, 352, 330, 378, 378, 333, 333, 340, 66, 308, 340, 66, 308, 366, 366, 311, 311, 367, 275, 363, 367, 275, 363, 304, 275, 363, 304, 275, 363, 379, 83, 375, 379, 83, 375, 353, 333, 330, 353, 333, 330, 330, 330, 378, 378, 333, 333]


In [27]:
print(len(network_output))

65680


In [28]:
network_input = np.reshape(network_input,(n_patterns,sequence_length,1))
print(network_input.shape)

(65680, 50, 1)


In [0]:
# Normalize our data [0,1]
normalized_network_input = network_input/float(n_vocab)
#print(normalized_network_input)

In [30]:
# Converting the y values into one hot vectors
network_output = np_utils.to_categorical(network_output)
print(network_output.shape)

(65680, 381)


In [31]:
print(normalized_network_input.shape)
print(network_output.shape)

(65680, 50, 1)
(65680, 381)


## Loading the trained LSTM model

In [0]:
model = load_model("model.hdf5")

## Making the predictions on the model

In [0]:
sequence_length = 50
network_input = []
for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length] # in the string format
    network_input.append([ele_int[ch] for ch in seq_in])

In [0]:
# Taking a random start sequence to generate the next sequence
start = np.random.randint(len(network_input))
pattern = network_input[start]
prediction_output = []
int_to_ele = dict((idx,ele)for idx,ele in enumerate(snotes))

for i in range(200):
    prediction_input = np.reshape(pattern,(1,sequence_length,1))
    prediction_input = prediction_input/float(n_vocab)
    
    prediction = model.predict(prediction_input)
    idx = np.argmax(prediction)
    prediction_output.append(int_to_ele[idx])
    
    pattern.append(idx)
    pattern = pattern[1:]    

In [114]:
print(prediction_output)

['A4', '0+5', 'C5', 'A4', 'E5', 'C5', 'B4', 'D5', 'E5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'F#5', 'C5', 'A4', '0+5', 'C5', 'A4', 'E5', 'E3', 'C5', 'B2', 'B4', 'C3', 'D5', 'G#2', 'E5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'F#5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'E5', '4+9', 'C5', 'B4', '7', 'D5', 'E5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'F#5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'E5', '4+9', 'C5', 'B4', '7', 'D5', '9+0', '5', '4+7', '2+5', '7', '0+4', '11+2', '4', '0+4', 'E4', '2+5', '11', '11+2', '5', '7+11', '7', '9+0', '9', '11', '0', '4', 'A4', '5', 'F4', 'E5', 'A4', 'D5', '7', 'C5', 'A4', 'B4', '4', 'G4', 'C5', 'E4', 'D5', '11', 'G4', 'B4', '5', 'E4', 'G4', '7', 'E4', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '2+7', '4+9', '4+9', '4+9', '4+9', '4+9', '2+7', 'E4', '4+9', 'A4', 'B4', 'C5', '4+9', 'B4', 'A4', 'E4', '4+9', 'C4', 'B3', '4+9', 'C4', 'A3', '4+9', 'C4', 'B3', '7', 'C4', 'D4', '5', 'E4', 'C4', '5', 'E4', 'D4', '4', 'E4', 'F4'

## Creating a MIDI file to play the music generated by the model

In [0]:
output_notes = []
offset = 0

for pattern in prediction_output:
    #if the pattern is a chord, then
    if ('+' in pattern) or pattern.isdigit():
        all_notes = pattern.split("+")
        temp_notes = []
        #print(all_notes)
        for current_note in all_notes:
          new_note = note.Note(int(current_note))
          new_note.storedInstrument = instrument.Piano()
          temp_notes.append(new_note)
        new_chord = chord.Chord(temp_notes) # It creates a chord with the help of the list of the notes
        new_chord.storedInstrument = instrument.Piano()
        output_notes.append(new_chord)
        
    else:
        #if the pattern is a note,then
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

In [127]:
print(output_notes)

[<music21.note.Note A>, <music21.chord.Chord C F>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note E>, <music21.note.Note C>, <music21.note.Note B>, <music21.note.Note D>, <music21.note.Note E>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Note A>, <music21.chord.Chord C F>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note F#>, <music21.note.Note C>, <music21.note.Note A>, <music21.chord.Chord C F>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note E>, <music21.note.Note E>, <music21.note.Note C>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note C>, <music21.note.Note D>, <music21.note.Note G#>, <music21.note.Note E>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Note A>, <music21.chord.Chord C F>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note F#>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Not

In [128]:
midi_stream = stream.Stream(output_notes)
midi_stream.write("midi","/content/drive/My Drive/MLplayground/output1.mid")

'/content/drive/My Drive/MLplayground/output1.mid'

In [129]:
midi_stream.show('midi')